# Preprocessing

In [24]:
import pandas as pd

df_dev = pd.read_csv("fall_project_dataset/development.csv", index_col=0)
df_eval = pd.read_csv("fall_project_dataset/evaluation.csv", index_col=0)

## Reduce the cardinality of OCCP

In [25]:
import csv

# Create a dictionary from the OCCP code to the text representation
reader = csv.reader(open('produced_documents/occp_to_string.csv', 'r'), delimiter=';')
next(reader, None) # Skip the headers

occp_to_string = {}

for row in reader:
   k, v = row
   k = int(k)
   occp_to_string[k] = v

# Map the OCCP column to its text values
df_dev["OCCP"] = df_dev["OCCP"].map(occp_to_string)
df_eval["OCCP"] = df_eval["OCCP"].map(occp_to_string)

# Keep only the first 3 characters 
df_dev["OCCP"] = df_dev["OCCP"].apply(lambda occp : occp[0:3])
df_eval["OCCP"] = df_eval["OCCP"].apply(lambda occp : occp[0:3])

## Get the minutes range from JWAP and JWDP and calculate the possible range of JWMNP

In [26]:
reader = csv.reader(open('produced_documents/JWDP.csv', 'r'))
next(reader, None) # Skip the headers

jwdp_begin = {}
jwdp_end = {}

for row in reader:
   k, b, e = row
   k = int(k)
   jwdp_begin[k] = int(b)
   jwdp_end[k] = int(e)

reader = csv.reader(open('produced_documents/JWAP.csv', 'r'))
next(reader, None) # Skip the headers

jwap_begin = {}
jwap_end = {}

for row in reader:
   k, b, e = row
   k = int(k)
   jwap_begin[k] = int(b)
   jwap_end[k] = int(e)

# Map the JWDP column to extract minimum and maximum departure time in minutes
df_dev["JWDP_B"] = df_dev["JWDP"].map(jwdp_begin)
df_dev["JWDP_E"] = df_dev["JWDP"].map(jwdp_end)

df_eval["JWDP_B"] = df_eval["JWDP"].map(jwdp_begin)
df_eval["JWDP_E"] = df_eval["JWDP"].map(jwdp_end)

# Map the JWAP column to extract minimum and maximum arrival time in minutes
df_dev["JWAP_B"] = df_dev["JWAP"].map(jwap_begin)
df_dev["JWAP_E"] = df_dev["JWAP"].map(jwap_end)

df_eval["JWAP_B"] = df_eval["JWAP"].map(jwap_begin)
df_eval["JWAP_E"] = df_eval["JWAP"].map(jwap_end)

# Add two columns for the expected JWMNP range
df_dev["JWMNP_B"] = df_dev["JWAP_B"] - df_dev["JWDP_B"]
df_dev["JWMNP_E"] = df_dev["JWAP_E"] - df_dev["JWDP_E"]
df_dev["JWMNP_B_E"] = (df_dev["JWAP_E"] - df_dev["JWDP_B"])
df_dev["JWMNP_E_B"] = (df_dev["JWAP_B"] - df_dev["JWDP_E"])

df_eval["JWMNP_B"] = df_eval["JWAP_B"] - df_eval["JWDP_B"]
df_eval["JWMNP_E"] = df_eval["JWAP_E"] - df_eval["JWDP_E"]
df_eval["JWMNP_B_E"] = (df_eval["JWAP_E"] - df_eval["JWDP_B"])
df_eval["JWMNP_E_B"] = (df_eval["JWAP_B"] - df_eval["JWDP_E"])

## Drop the original JWDP and JWAP columns

In [27]:
df_dev = df_dev[["OCCP", "PINCP", "WKHP", "SEX", "JWDP_B", "JWDP_E", "JWAP_B", "JWAP_E", "JWMNP_B", "JWMNP_E", "JWMNP_B_E", "JWMNP_E_B", "JWMNP"]]
df_eval = df_eval[["OCCP", "PINCP", "WKHP", "SEX", "JWDP_B", "JWDP_E", "JWAP_B", "JWAP_E", "JWMNP_B", "JWMNP_E", "JWMNP_B_E", "JWMNP_E_B"]]
df_dev

,OCCP,PINCP,WKHP,SEX,JWDP_B,JWDP_E,JWAP_B,JWAP_E,JWMNP_B,JWMNP_E,JWMNP_B_E,JWMNP_E_B,JWMNP
Id,,,,,,,,,,,,,
0,CLN,60000.0,60.0,1.0,330,334,405,409,75,75,79,71,75.0
1,SAL,41900.0,35.0,1.0,780,789,790,794,10,5,14,1,10.0
2,SAL,278000.0,50.0,2.0,540,544,545,549,5,5,9,1,5.0
3,ENG,99000.0,40.0,1.0,510,514,545,549,35,35,39,31,35.0
4,SAL,43000.0,40.0,1.0,345,349,350,354,5,5,9,1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104637,PRD,15000.0,40.0,1.0,420,424,435,439,15,15,19,11,15.0
104638,OFF,30000.0,50.0,1.0,270,279,295,299,25,20,29,16,25.0
104639,SAL,52000.0,40.0,2.0,500,504,520,524,20,20,24,16,20.0


# Train-Test split

In [28]:
from sklearn.model_selection import train_test_split

X = df_dev.drop(columns=["JWMNP"])
y = df_dev["JWMNP"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a simple model

## Prepare the pipeline

In [29]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_features = ['PINCP', 'WKHP', 'JWMNP_B', 'JWMNP_E', 'JWMNP_B_E', 'JWMNP_E_B']
numeric_transformer = StandardScaler()

categorical_features = ['OCCP', 'SEX', 'JWDP_B', 'JWDP_E', 'JWAP_B', 'JWAP_E']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

# pipe = Pipeline(steps=[('preprocessor', preprocessor),
#                        ('regressor', RandomForestRegressor(n_jobs=-1))])

# pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9968004116711765

In [31]:
from sklearn.metrics import r2_score

print(pipe.score(X_test, y_test))
y_pred = pipe.predict(X_test)
print(r2_score(y_test, y_pred))

0.9968004116711765
0.9968004116711765


# Create the submission file

In [33]:
# TODO Check if the preprocessing to drop the columns is needed for df_eval
y_pred = pipe.predict(df_eval)

data = list(zip(df_eval.index, y_pred))

# Save the data to a CSV file
with open('submissions/submission_2.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Id', 'Predicted'])  # Header row
    csvwriter.writerows(data)